In [ ]:
%matplotlib notebook
import pandas as pd
import pandasql as pds
from pandasql import sqldf
import plotly.graph_objects as go
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 50)
pd.set_option('display.min_rows', 50)
import seaborn as sns
from dateutil.relativedelta import relativedelta

In [ ]:

# Load the data
big_lot_table_8 = pd.read_csv(r"C:\Users\MichaelDiFelice\Documents\Sanofi\MM\DATA\Flatiron\Big LoT Table\Big LoT Table.csv")

# Convert STARTDATE to datetime format
big_lot_table_8['STARTDATE'] = pd.to_datetime(big_lot_table_8['STARTDATE'])

# Define the reference date
reference_date = pd.to_datetime('2023-07-31')

# Filter the rows based on ISFIRSTTREATMENT
first_treatment_data = big_lot_table_8[big_lot_table_8['ISFIRSTTREATMENT'] == True]

# Calculate the exact difference in months between two dates
def months_difference(start_date, end_date):
    delta = relativedelta(end_date, start_date)
    return delta.years * 12 + delta.months

# Calculate the lookback in 12-month increments (up to 48 months)
def lookback_periods(months):
    if months < 12:
        return "0-12 months"
    elif months < 24:
        return "12-24 months"
    elif months < 36:
        return "24-36 months"
    elif months < 48:
        return "36-48 months"
    else:
        return "more than 48 months"

# Create a copy of the filtered data to avoid SettingWithCopyWarning
first_treatment_data_copy = first_treatment_data.copy()

# Calculate the exact months since start for each row
first_treatment_data_copy['exact_months_since_start'] = first_treatment_data_copy['STARTDATE'].apply(lambda x: months_difference(x, reference_date))

# Apply the lookback_periods function to determine the lookback based on the exact month difference
first_treatment_data_copy['exact_lookback'] = first_treatment_data_copy['exact_months_since_start'].apply(lookback_periods)

data_final = first_treatment_data_copy[first_treatment_data['LINE_ZERO_FLAG']==0]




In [ ]:
def bundle_regimen(row):
    # Line-based conditions
    if row['LINENUMBER'] == 1:
        # Transplant and regimen conditions within Line 1
        if row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] == "DVRd":
            return "DVRd"
        elif row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] == "DVTd":
            return "DVTd"
        elif row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] == "VCd":
            return "VCd"
        elif row['TRANSPLANT_FLAG'] == 1 and row['REGIMEN'] in ["VRd", "VR", "Vd",]:
            return "VRd"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] in ['D mono','DKd','DKRd','DPd','DVd','DVMp','DVTd','Other D']:
            return "D-other"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'DRd':
            return "DRd"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'DVRd':
            return "DVRd"
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] in ['Kd','Pd','Vd']:
            return 'Other doublet'
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] in ['EPd','ERd','IRd','IsaKd','IsaPd','KPd','KRd','PCd','PVd','VCd','VMp','VTd']:
            return 'Other triplet'
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'Rd':
            return 'Rd'
        elif row['TRANSPLANT_FLAG'] == 0 and row['REGIMEN'] == 'VRd':
            return 'VRd' 

    elif row['LINENUMBER'] == 2:
        # Transplant and regimen conditions within Line 2
        if row['REGIMEN'] in ['D mono','DKRd','DVMp','DVRd','DVTd','Other D']:
            return "D-other"
        elif row['REGIMEN'] == "DKd":
            return "DKd"
        elif row['REGIMEN'] == "DPd":
            return "DPd"
        elif row['REGIMEN'] == "DRd":
            return "DRd"
        elif row['REGIMEN'] == "DVd":
            return "DVd"
        elif row['REGIMEN'] == "IsaKd":
            return "IsaKd"
        elif row['REGIMEN'] == "IsaPd":
            return "IsaPd"
        elif row['REGIMEN'] == 'Rd':
            return 'Rd'
        elif row["REGIMEN"] in ['ERd','IRd','KRd','VRd']:
            return 'R-Triplet'


    # Default return value if none of the conditions are met
    return "Other"

# Apply the function to create the 'bundled_regimen' column
data_final['BUNDLED_REGIMEN'] = data_final.apply(bundle_regimen, axis=1)



In [ ]:

data_sorted = data_final.drop_duplicates(subset=['PATIENTID', 'LINENUMBER'])

# Sort the data by PATIENTID and LINENUMBER to ensure the order
data_sorted = data_sorted.sort_values(by=["PATIENTID", "LINENUMBER"])

# Shift the REGIMEN column to get the next regimen for each patient and line
data_sorted["NEXT_REGIMEN"] = data_sorted.groupby("PATIENTID")["REGIMEN"].shift(-1)
data_sorted['PREVIOUS_REGIMEN'] = data_sorted.groupby("PATIENTID")["REGIMEN"].shift(1)
# Fill NaN values in "NEXT_REGIMEN" with "No Advancement"
data_sorted["NEXT_REGIMEN"].fillna("No Advancement", inplace=True)
data_sorted["PREVIOUS_REGIMEN"].fillna("", inplace=True)


# If NEXT_REGIMEN is 'No Advancement', set NEXT_LINENUMBER to the same as LINENUMBER
# Otherwise, shift the LINENUMBER column to get the next line number for each patient
data_sorted["NEXT_LINENUMBER"] = data_sorted.apply(
    lambda row: row["LINENUMBER"] if row["NEXT_REGIMEN"] == "No Advancement" else row["LINENUMBER"] + 1, axis=1
)

data_sorted["START_YEAR"] = data_sorted['exact_lookback']

# Filter out rows where the next line is not consecutive (e.g., line 1 followed by line 3) or 'No Advancement'
data_sorted = data_sorted[(data_sorted["LINENUMBER"] + 1 == data_sorted["NEXT_LINENUMBER"]) | (data_sorted["NEXT_REGIMEN"] == "No Advancement")]

# For our current task, we only need certain columns
data = data_sorted[["PATIENTID", "LINENUMBER", "REGIMEN","PREVIOUS_REGIMEN", "NEXT_REGIMEN", "LEN_FLAG", "CD38_FLAG", "CD38_EXPOSED_FLAG", "TRANSPLANT_FLAG", "START_YEAR","LEN_REFRACTORY_FLAG","NEXT_LINENUMBER"]]
data = data[data['LINENUMBER'].isin([1, 2, 3, 4])]

# Concatenate LINENUMBER to REGIMEN and NEXT_REGIMEN columns
data['REGIMEN'] = data['REGIMEN'].astype(str) + data['LINENUMBER'].astype(str)
data['NEXT_REGIMEN'] = data['NEXT_REGIMEN'].astype(str) + (data['LINENUMBER'] + 1).astype(str)
data['PREVIOUS_REGIMEN'] = data['PREVIOUS_REGIMEN'].astype(str) + (data['LINENUMBER'] - 1).astype(str)
data['PREVIOUS_REGIMEN'] = data['PREVIOUS_REGIMEN'].str.replace('0', '')

# Turn the data into a cvs in the Modified Data folder
data.to_csv(r"C:\Users\MichaelDiFelice\Documents\Sanofi\Python\Dashboard\Data\Modified Data\modified_sankey.csv", index=False)